# Prototype


### First data proccesing into pandas or numpy 

In [16]:
import pandas as pd 

df = pd.read_excel("annotations.xlsx",index_col=0 )
df.drop(df.columns.difference(["text","type"]), 1, inplace=True)
distrub = df['type'].value_counts(normalize=True)
one_hot = pd.get_dummies(df["type"])
df = df.drop("type",axis = 1)
df = df.join(one_hot)

print(df)



                                                    text  center  left  right
0      The transgender effort to suppress any recogni...       0     0      1
1      Radical Virginia Citizens Defense League has o...       0     1      0
2      Miller is the architect of President Donald Tr...       0     1      0
3      The House Democrats’ 1,400-page coronavirus re...       0     0      1
4      A specter is haunting the West; our elites see...       0     0      1
...                                                  ...     ...   ...    ...
22466  Oregon first graders could attend gun safety c...       1     0      0
22467  But President Donald Trump and his campaign ad...       0     1      0
22468  The normalization effort included softened rhe...       1     0      0
22469  And so, while demonstrations began to spread a...       0     1      0
22470  House and Senate Democrats are responding to t...       0     0      1

[17775 rows x 4 columns]


C:\Users\mrimb\AppData\Local\Temp/ipykernel_17464/3989888564.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(["text","type"]), 1, inplace=True)


### Exploratory data analysis 

In [19]:
#numerical data anaylisis 
center = df['center'].value_counts()
left = df['left'].value_counts()
right = df['right'].value_counts()
print("Percentage Distrubtuion")
print(distrub * 100) 

Percentage Distrubtuion
left      40.753868
right     40.686357
center    18.559775
Name: type, dtype: float64
